# Tool-Augmented LangGraph RAG

This notebook extends the base LangGraph pipeline with **tool routing**. A controller decides whether to answer using our uploaded PDFs or to call an external search tool (Tavily) when internal knowledge is insufficient. Docstrings describe when each tool should be chosen so the language model understands *why/when/how* to invoke them.


In [ ]:
import os  # Filesystem helper to list resume PDFs.
from typing import TypedDict  # Keeps LangGraph state strongly typed.

from IPython.display import Image, display  # Used later to render the graph PNG inline.
from langchain_community.document_loaders import PyPDFLoader  # Same loader stack as other notebooks.
from langchain_text_splitters import RecursiveCharacterTextSplitter  # Creates overlap-aware chunks.
from langchain_huggingface import HuggingFaceEmbeddings  # Provides MiniLM embeddings.
from langchain_community.vectorstores import FAISS  # In-memory FAISS index.
from langchain_ollama import OllamaLLM  # Local LLM that powers both router + answer nodes.
from langchain_core.prompts import ChatPromptTemplate  # Prompt templating utility for router + answer.
from langchain_community.tools.tavily_search import TavilySearchResults  # Web-search tool (requires TAVILY_API_KEY).

from langgraph.graph import StateGraph, START, END  # LangGraph primitives.



c:\Users\Asus\Desktop\all projects & learnings\llm-rag-fastapi-study\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Load and Embed Internal Documents
We keep the ingestion identical to the previous notebooks so the only new behavior comes from the tool-routing logic.


In [ ]:
DOCS_FOLDER = r"C:\\Users\\Asus\\Documents\\all_doc"  # Reuse the same resume/document directory.

documents = []  # Each entry becomes a LangChain Document created per PDF page.
for file in os.listdir(DOCS_FOLDER):  # Loop across directory contents.
    if file.lower().endswith(".pdf"):  # Only parse PDFs.
        loader = PyPDFLoader(os.path.join(DOCS_FOLDER, file))  # Keeps metadata like source filename + page.
        documents.extend(loader.load())  # Append every page Document into the corpus list.

len(documents)  # Gives visibility into how many pages will move through chunking.



In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Maintains parity with other notebooks.
    chunk_overlap=10,  # Protects sentence continuity.
)
chunks = splitter.split_documents(documents)  # Expand into chunked Document objects.
print(f"Split into {len(chunks)} chunks")

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # MiniLM embeddings.
vector_store = FAISS.from_documents(chunks, embedding)  # Store embeddings in FAISS for similarity search.
retriever = vector_store.as_retriever(search_kwargs={"k": 10})  # Search top-10 by cosine similarity.
print(f"Vector store holds {vector_store.index.ntotal} rows")


## 2. Define Tooling Layer
We expose two tools:

1. **InternalDocsTool** – pulls context from the FAISS retriever. Docstring explains it should be used for resume-specific questions.
2. **TavilySearchTool** – hits the public web (needs `TAVILY_API_KEY`). Docstring tells the LLM to call it when users ask for fresh/external info.

LangChain/agents rely heavily on docstrings/descriptions to decide which tool to call, so we surface the same concept here.


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env file automatically

In [ ]:
tavily = TavilySearchResults(
    max_results=3  # Limit API usage + keep responses concise.
)  # Requires `TAVILY_API_KEY` to be set in the environment before execution.


def internal_docs_tool(question: str) -> str:
    """Use this tool when the answer likely exists inside the uploaded resumes or project PDFs."""

    docs = retriever.invoke(question)  # Hit FAISS for semantic matches.
    return "\n\n".join(doc.page_content for doc in docs)  # Return concatenated passages.


def tavily_search_tool(question: str) -> str:
    """Use this tool for real-time/company-wide facts that are NOT covered by the uploads."""

    results = tavily.invoke({"query": question})  # Returns JSON-friendly search summaries.
    return "\n".join(hit["content"] for hit in results["results"])  # Collapse into plain text for the LLM.



## 3. LangGraph State + Router Prompt
The router LLM reads both docstrings so it understands *why* each tool exists. It outputs one of `internal_docs`, `web_search`, or `hybrid` (use both).


In [ ]:
class ToolRAGState(TypedDict, total=False):
    """Shared state for the tool-enabled graph."""

    question: str  # Original user query.
    selected_tool: str  # Router decision: internal_docs / web_search / hybrid.
    internal_context: str  # Text retrieved from FAISS.
    web_context: str  # Text returned by Tavily.
    answer: str  # Final LLM response.

TOOL_DOCS = {
    "internal_docs": internal_docs_tool.__doc__,
    "web_search": tavily_search_tool.__doc__,
}

tool_routing_prompt = ChatPromptTemplate.from_template(
    """You are a routing model that decides which tool to call before answering a question.
Available tools:

- internal_docs: {internal_docs_doc}
- web_search: {web_search_doc}

Return one of: internal_docs, web_search, hybrid.
Question: {question}
"""
)

router_llm = OllamaLLM(model="llama3.2")  # Reuse the same local model for routing decisions.
answer_llm = OllamaLLM(model="llama3.2")  # Separate instance for clarity (could be shared).



In [ ]:
def router_node(state: ToolRAGState) -> ToolRAGState:
    """Ask the LLM which tool(s) should run given the question + docstrings."""

    decision = router_llm.invoke(
        tool_routing_prompt.format(
            question=state["question"],
            internal_docs_doc=TOOL_DOCS["internal_docs"],
            web_search_doc=TOOL_DOCS["web_search"],
        )
    ).strip().lower()
    normalized = decision if decision in {"internal_docs", "web_search", "hybrid"} else "internal_docs"
    return {**state, "selected_tool": normalized}


def internal_retrieve_node(state: ToolRAGState) -> ToolRAGState:
    """Populate `internal_context` when the router requested internal knowledge."""

    context = internal_docs_tool(state["question"])
    return {**state, "internal_context": context}


def web_search_node(state: ToolRAGState) -> ToolRAGState:
    """Populate `web_context` via Tavily when the router asked for external info."""

    context = tavily_search_tool(state["question"])
    return {**state, "web_context": context}


def answer_node(state: ToolRAGState) -> ToolRAGState:
    """Combine whichever contexts exist and ask the answering LLM to respond."""

    context_sections = []
    if "internal_context" in state:
        context_sections.append("INTERNAL DOCS\n" + state["internal_context"])
    if "web_context" in state:
        context_sections.append("WEB SEARCH\n" + state["web_context"])

    prompt = f"""Use the consolidated evidence below to answer the question.
{''.join(section + '\n\n' for section in context_sections)}
Question: {state['question']}
"""
    answer = answer_llm.invoke(prompt)
    return {**state, "answer": answer}



## 4. Wire the LangGraph with Conditional Edges
Conditional edges let us branch based on the router decision. Hybrid mode simply runs internal retrieval first, then falls through to web search before synthesis.


In [ ]:
def route_from_router(state: ToolRAGState) -> str:
    """Return the next node label right after the router."""

    return state["selected_tool"]


def route_after_internal(state: ToolRAGState) -> str:
    """Decide whether to run web search next (hybrid) or go straight to answering."""

    return "web_search" if state["selected_tool"] == "hybrid" else "answer"


graph_builder = StateGraph(ToolRAGState)

graph_builder.add_node("router", router_node)
graph_builder.add_node("internal", internal_retrieve_node)
graph_builder.add_node("web", web_search_node)
graph_builder.add_node("answer", answer_node)

graph_builder.add_edge(START, "router")

graph_builder.add_conditional_edges(
    "router",
    route_from_router,
    {
        "internal_docs": "internal",
        "web_search": "web",
        "hybrid": "internal",
    },
)

graph_builder.add_conditional_edges(
    "internal",
    route_after_internal,
    {
        "web_search": "web",
        "answer": "answer",
    },
)

graph_builder.add_edge("web", "answer")
graph_builder.add_edge("answer", END)

tools_graph = graph_builder.compile()



## 5. Run Sample Questions
Try one question that internal docs can answer and another that requires web search to demonstrate the router + docstrings at work.


In [ ]:
questions = [
    "list the projects done in meta and teksystems",  # Should route to internal_docs.
    "latest news about teksystems partnerships",  # Likely requires web_search.
]

for question in questions:
    result = tools_graph.invoke({"question": question})
    print(f"Question: {question}")
    print(f"Selected tool: {result.get('selected_tool')}")
    print(f"Answer: {result.get('answer')}\n")



## 6. Visualize Tool Routing
As before, we export Mermaid text + PNG so the control flow is easy to inspect.


In [ ]:
mermaid_text = tools_graph.get_graph().draw_mermaid()  # Text version for docs.
print(mermaid_text)

with open("toolslanggraph.mmd", "w", encoding="utf-8") as f:
    f.write(mermaid_text)

mermaid_png = tools_graph.get_graph().draw_mermaid_png()
with open("toolslanggraph.png", "wb") as f:
    f.write(mermaid_png)

display(Image(mermaid_png))


## Takeaways
- Docstrings feed the router prompt, so the LLM knows *when* to call each tool.
- Conditional LangGraph edges make it easy to extend the flow (add evaluators, safety checks, etc.).
- Both the internal FAISS retriever and Tavily outputs remain visible in the final state, enabling detailed telemetry or UI surfacing.
